In [38]:
import redis
import re
import mysql.connector

In [39]:
def connection_db(nameDB):
    mydb = mysql.connector.connect(
        host = 'localhost',
        user = 'root',
        password = '',
        database = nameDB
        )
    curr = mydb.cursor()
    return curr, mydb

In [40]:
def sorting_emiten(reType5, emit):
    for emiten in range(len(reType5)):
        symbol = re.split('\,', str(reType5[emiten]))
        symbol[4] = re.sub("\'","", str(symbol[4]))
        symbol[4] = re.sub("\s","", str(symbol[4]))
        emit.append(symbol[4])
    emit.sort()
    emit = daftar_emiten(emit)
    return emit

In [41]:
def daftar_emiten(emit):
    data = []
    get_list = ''
    for em in (emit) :
        if (em != get_list) :
            get_list = em
            data.append(em)
    return data

In [42]:
def append_list(a,b) :
    for i in range(len(b)) :
        a.append(b[i])
    return a

In [43]:
def reg(data) :
    
    data = re.sub('\s','', str(data))
    data = re.sub('\"','', str(data))
    return data

In [44]:
def remove_list(a,b) :
    for i in range(len(b)) :
        a.remove(b[i])
    return a

In [45]:
def bagiData(data) :
    La = []
    Lb = []
    for i in range (len(data)) :
        a,b = data[i]
        La.append(a)
        Lb.append(b)
    return La,Lb

In [46]:
def insert_open(dataRT, a, b, listEM) :
    n = len(dataRT)
    emit = []
    for em in listEM :
        for i in range (n) :
            a[i] = re.sub('\'','', str(a[i]))
            b[i] = re.sub('\'','', str(b[i]))
            if (em == a[i]) :
                list = em,b[i]
                emit.append(em)
                dataRT.append(list)
                break
    
    print("banyak emiten : ", (len(listEM)))
    print("emiten terisi : ", (len(emit)))
    print("")

    emit = remove_list(listEM, emit)
    return dataRT, emit

In [47]:
def daftar_open_price(reType1, reType2, nEmiten):
    data, i_data1, i_data2 = kondisi_rt1(reType1,6,9)
    i_listOP, i_listEm = insert_open(i_data, i_data1, i_data2, nEmiten)
    
#     data, i_data1, i_data2 = kondisi_rt2(reType2,6,9)
#     j_listOP, j_listEm = insert_open(j_data, j_data1, j_data2, i_listEm)
    
#     data = append_list(i_listOP, j_listEm)
#     return data, j_listOP

In [48]:
def kondisi_rt1(result, data1, data2):
    j_data = []
    i_data1 = []
    i_data2 = []
    for j in result :
        row = re.split('\,',str(j))
        if (int(row[1]) < 90000) : 
            row[data1] = reg(row[data1])
            row[data2] = reg(row[data2])
            data = (row[data1],row[data2])
            i_data1.append(row[data1])
            i_data2.append(row[data2])
            j_data.append(data)
    return j_data, i_data1, i_data2

In [49]:
def kondisi_rt2(result, data1, data2):
    j_data = []
    i_data1 = []
    i_data2 = []
    for j in result :
        row = re.split('\,',str(j))
        if (int(row[1]) > 85000) and (int(row[5]) ==0): 
            row[data1] = reg(row[data1])
            row[data2] = reg(row[data2])
            data = (row[data1],row[data2])
            i_data1.append(row[data1])
            i_data2.append(row[data2])
            j_data.append(data)
    return j_data, i_data1, i_data2

In [50]:
def filter_redis_paket(r, reType1, reType2, reType5, nameDB):
    reType=[]
    for i in range (9999) :
        go = r.get(i+2)
        if (go == None):
            break
        elif (go !=b'"') and (go != b"'") :
            data=go.decode('utf-8','ignore')
            data = re.sub('\s','', str(data))
            rowData = re.split('\|',str(data))
            if (rowData[4]=='1') :
                reType1.append(rowData[1:20])
            elif (rowData[4]=='2') :
                reType2.append(rowData[1:22])
            elif (rowData[4]=='5') :
                reType5.append(rowData[1:24])
    return  reType1, reType2, reType5

In [51]:
def main() :
    reType1=[]
    reType2=[]
    reType5=[]
    emiten=[]
    
    r = redis.Redis()
    nameDB='PaketData'
    reType1, reType2, reType5 = filter_redis_paket(r, reType1, reType2, reType5, nameDB)
    
    list_emiten = sorting_emiten(reType5,emiten) #594
#     print(len(list_emiten))
    
    list_open, list_emit = daftar_open_price(reType1, reType2, list_emiten)
    
    print('Data sisa = ',len(list_emit))


In [52]:
main()

ValueError: invalid literal for int() with base 10: " '120000'"